# 📋 레시피 데이터 처리 및 통합

이 노트북에서는 식품안전나라 API에서 레시피 데이터를 수집하고,
여러 개의 레시피 순서 컬럼을 하나로 통합하여 정리합니다.

## 🎯 목표
1. 식품안전나라 API에서 레시피 데이터 수집
2. MANUAL01~MANUAL20 컬럼을 하나의 'recipe_steps' 컬럼으로 통합
3. 레시피 순서를 '1. 씻는다, 2. 손질한다, 3. 썬다' 형태로 정리
4. 깔끔한 CSV 파일로 저장

## 1. 📊 데이터 수집 및 기본 처리

In [111]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import re
import requests
import json
import os
from time import sleep

In [112]:
def fetch_all_recipe_data():
    """
    식품안전나라 API에서 모든 레시피 데이터를 수집하는 함수
    """
    API_KEY = "8434c606f68343abaeb9"
    SERVICE_ID = "COOKRCP01"
    DATA_TYPE = "json"
    BATCH_SIZE = 1000
    
    all_recipes = []
    start_idx = 1
    
    print("🔄 API에서 레시피 데이터 수집을 시작합니다...")
    
    while True:
        end_idx = start_idx + BATCH_SIZE - 1
        url = f"http://openapi.foodsafetykorea.go.kr/api/{API_KEY}/{SERVICE_ID}/{DATA_TYPE}/{start_idx}/{end_idx}"
        
        print(f"📥 요청 중: {start_idx} ~ {end_idx}")
        
        try:
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                
                if 'COOKRCP01' in data:
                    if start_idx == 1:
                        total_count = data['COOKRCP01']['total_count']
                        print(f"📊 전체 데이터 개수: {total_count}개")
                    
                    if 'row' in data['COOKRCP01']:
                        recipes = data['COOKRCP01']['row']
                        all_recipes.extend(recipes)
                        print(f"✅ 현재까지 수집된 데이터: {len(all_recipes)}개")
                        
                        if len(recipes) < BATCH_SIZE:
                            print("🎉 마지막 배치 완료")
                            break
                    else:
                        print("⚠️ 더 이상 데이터가 없습니다.")
                        break
                else:
                    print("❌ 예상하지 못한 응답 구조:", data)
                    break
            else:
                print(f"❌ API 요청 실패: {response.status_code}")
                break
                
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            break
        
        start_idx += BATCH_SIZE
        sleep(0.1)  # API 호출 간 대기
    
    print(f"\n🎯 총 {len(all_recipes)}개의 레시피 데이터 수집 완료!")
    return all_recipes

# 데이터 수집 실행
recipes_data = fetch_all_recipe_data()

🔄 API에서 레시피 데이터 수집을 시작합니다...
📥 요청 중: 1 ~ 1000
📊 전체 데이터 개수: 1136개
✅ 현재까지 수집된 데이터: 1000개
📥 요청 중: 1001 ~ 2000
✅ 현재까지 수집된 데이터: 1136개
🎉 마지막 배치 완료

🎯 총 1136개의 레시피 데이터 수집 완료!


In [113]:
# DataFrame 생성 및 기본 정보 확인
if recipes_data:
    df = pd.DataFrame(recipes_data)
    
    print("📋 DataFrame 생성 완료!")
    print(f"📏 데이터 크기: {df.shape}")
    print(f"📊 총 컬럼 수: {len(df.columns)}개")
    
    # 현재 컬럼 순서 확인
    print(f"\n🔍 현재 컬럼 순서 (처음 10개): {list(df.columns)[:10]}")
    
    # 주요 컬럼 미리보기
    key_columns = ['RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG']
    print(f"\n🔍 주요 데이터 미리보기:")
    print(df[key_columns].head())
    
    # 레시피 단계 컬럼 확인
    manual_cols = [col for col in df.columns if col.startswith('MANUAL') and not col.startswith('MANUAL_IMG')]
    print(f"\n📝 레시피 단계 컬럼 ({len(manual_cols)}개): {manual_cols[:6]}...")
    
else:
    print("❌ 수집된 데이터가 없습니다.")
    df = pd.DataFrame()

📋 DataFrame 생성 완료!
📏 데이터 크기: (1136, 55)
📊 총 컬럼 수: 55개

🔍 현재 컬럼 순서 (처음 10개): ['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14']

🔍 주요 데이터 미리보기:
           RCP_NM RCP_WAY2 RCP_PAT2 INFO_ENG
0       새우 두부 계란찜       찌기       반찬      220
1        부추 콩가루 찜       찌기       반찬      215
2       방울토마토 소박이       기타       반찬       45
3  순두부 사과 소스 오이무침       기타       반찬       75
4       사과 새우 북엇국      끓이기     국&찌개       65

📝 레시피 단계 컬럼 (20개): ['MANUAL20', 'MANUAL01', 'MANUAL08', 'MANUAL09', 'MANUAL06', 'MANUAL07']...


In [114]:
# DataFrame 컬럼 순서 정리
def reorder_dataframe_columns(df):
    """
    DataFrame의 컬럼을 논리적 순서로 재정렬하는 함수
    
    Args:
        df: 원본 DataFrame
        
    Returns:
        컬럼이 재정렬된 DataFrame
    """
    if df.empty:
        return df
    
    # 1. 기본 정보 컬럼들 (순서대로)
    basic_info = [
        'RCP_SEQ',        # 레시피 순번
        'RCP_NM',         # 레시피명
        'RCP_WAY2',       # 조리방법
        'RCP_PAT2',       # 요리종류
        'INFO_WGT',       # 중량
        'INFO_ENG',       # 칼로리
        'INFO_CAR',       # 탄수화물
        'INFO_PRO',       # 단백질
        'INFO_FAT',       # 지방
        'INFO_NA'         # 나트륨
    ]
    
    # 2. 재료 및 설명 컬럼들
    content_info = [
        'RCP_PARTS_DTLS', # 재료정보
        'RCP_NA_TIP',     # 나트륨 팁
        'HASH_TAG'        # 해시태그
    ]
    
    # 3. 이미지 컬럼들 (메인 이미지)
    main_images = [
        'ATT_FILE_NO_MAIN',  # 메인 이미지
        'ATT_FILE_NO_MK'     # 만들기 이미지
    ]
    
    # 4. 레시피 단계별 컬럼들 (MANUAL01~MANUAL20 + 해당 이미지)
    manual_columns = []
    for i in range(1, 21):
        manual_col = f'MANUAL{i:02d}'
        img_col = f'MANUAL_IMG{i:02d}'
        
        if manual_col in df.columns:
            manual_columns.append(manual_col)
        if img_col in df.columns:
            manual_columns.append(img_col)
    
    # 5. 기타 컬럼들 (위에서 분류되지 않은 것들)
    all_categorized = set(basic_info + content_info + main_images + manual_columns)
    other_columns = [col for col in df.columns if col not in all_categorized]
    
    # 최종 컬럼 순서 결정
    ordered_columns = []
    
    # 존재하는 컬럼만 추가
    ordered_columns.extend([col for col in basic_info if col in df.columns])
    ordered_columns.extend([col for col in content_info if col in df.columns])
    ordered_columns.extend([col for col in main_images if col in df.columns])
    ordered_columns.extend(other_columns)  # 기타 컬럼들
    ordered_columns.extend(manual_columns)  # 레시피 단계는 마지막에
    
    return df[ordered_columns]

# 컬럼 순서 정리 실행
if not df.empty:
    print("🔄 컬럼 순서를 논리적으로 정리하는 중...")
    print(f"정리 전 컬럼 순서 (처음 10개): {list(df.columns)[:10]}")
    
    df_reordered = reorder_dataframe_columns(df)
    
    print(f"✅ 컬럼 순서 정리 완료!")
    print(f"정리 후 컬럼 순서 (처음 15개): {list(df_reordered.columns)[:15]}")
    
    # 컬럼 카테고리별 개수 확인
    basic_cols = ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA']
    content_cols = ['RCP_PARTS_DTLS', 'RCP_NA_TIP', 'HASH_TAG']
    manual_cols = [col for col in df_reordered.columns if col.startswith('MANUAL')]
    
    print(f"\n📊 컬럼 구성:")
    print(f"- 기본정보: {len([c for c in basic_cols if c in df_reordered.columns])}개")
    print(f"- 내용정보: {len([c for c in content_cols if c in df_reordered.columns])}개") 
    print(f"- 레시피단계: {len(manual_cols)}개")
    print(f"- 전체: {len(df_reordered.columns)}개")
    
    # 원본 df 업데이트
    df = df_reordered
    
else:
    print("❌ DataFrame이 없습니다.")

🔄 컬럼 순서를 논리적으로 정리하는 중...
정리 전 컬럼 순서 (처음 10개): ['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14']
✅ 컬럼 순서 정리 완료!
정리 후 컬럼 순서 (처음 15개): ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'RCP_PARTS_DTLS', 'RCP_NA_TIP', 'HASH_TAG', 'ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK']

📊 컬럼 구성:
- 기본정보: 10개
- 내용정보: 3개
- 레시피단계: 40개
- 전체: 55개


## 2. 🔧 레시피 순서 통합 처리

In [115]:
# 빈 값들을 null로 치환하여 데이터 정리
def clean_empty_values(df):
    """
    DataFrame의 빈 문자열, 공백만 있는 값들을 null(NaN)로 치환
    """
    df_cleaned = df.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':  # 문자열 컬럼만 처리
            # 빈 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace('', np.nan)
            # 공백만 있는 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace(r'^\s*$', np.nan, regex=True)
    
    return df_cleaned

if not df.empty:
    print("🧹 데이터 정리 중...")
    df_cleaned = clean_empty_values(df)
    df = df_cleaned
    print("✅ 빈 값들을 null로 정리 완료!")
else:
    print("❌ DataFrame이 없습니다.")

🧹 데이터 정리 중...
✅ 빈 값들을 null로 정리 완료!


C:\Users\user\AppData\Local\Temp\ipykernel_66496\2353208295.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[col] = df_cleaned[col].replace('', np.nan)


In [116]:
def combine_recipe_steps(df):
    """
    MANUAL01~MANUAL20 컬럼들을 하나의 'recipe_steps' 컬럼으로 통합하는 함수
    
    Args:
        df: 원본 DataFrame
        
    Returns:
        통합된 recipe_steps 컬럼이 추가된 DataFrame
    """
    df_combined = df.copy()
    recipe_steps = []
    
    print("🔄 레시피 순서 통합 처리 중...")
    
    for idx in range(len(df)):
        steps = []
        step_number = 1
        
        # MANUAL01부터 MANUAL20까지 순서대로 확인
        for i in range(1, 21):
            manual_col = f'MANUAL{i:02d}'
            
            if manual_col in df.columns:
                step_text = df.iloc[idx][manual_col]
                
                # null이 아니고 의미있는 텍스트가 있는 경우만 추가
                if pd.notna(step_text) and str(step_text).strip():
                    # 기존 번호 제거 및 텍스트 정리
                    clean_text = str(step_text).strip()
                    # 앞의 숫자와 점 제거 (예: "1. ", "2. ")
                    clean_text = re.sub(r'^\d+\.\s*', '', clean_text)
                    # 뒤의 알파벳 제거 (예: "a", "b", "c")
                    clean_text = re.sub(r'[a-zA-Z]$', '', clean_text).strip()
                    
                    if clean_text:  # 정리 후에도 텍스트가 있으면
                        steps.append(f"{step_number}. {clean_text}")
                        step_number += 1
        
        # 모든 단계를 하나의 문자열로 합치기 (줄바꿈으로 구분)
        if steps:
            recipe_steps.append('\n'.join(steps))
        else:
            recipe_steps.append('')  # 빈 문자열
    
    # 새 컬럼 추가
    df_combined['recipe_steps'] = recipe_steps
    
    print(f"✅ recipe_steps 컬럼 생성 완료!")
    return df_combined

# 레시피 순서 통합 실행
if not df.empty:
    df_with_steps = combine_recipe_steps(df)
    
    # 결과 확인
    print(f"\n📋 첫 번째 레시피 예시 ({df_with_steps.iloc[0]['RCP_NM']}):")
    print("=" * 60)
    print(df_with_steps.iloc[0]['recipe_steps'])
    
    # 통계 정보
    total_recipes = len(df_with_steps)
    recipes_with_steps = (df_with_steps['recipe_steps'] != '').sum()
    print(f"\n📊 통계:")
    print(f"- 전체 레시피: {total_recipes:,}개")
    print(f"- 조리순서가 있는 레시피: {recipes_with_steps:,}개")
    print(f"- 조리순서 비율: {(recipes_with_steps/total_recipes)*100:.1f}%")
    
    # 원본 df 업데이트
    df = df_with_steps
    
else:
    print("❌ DataFrame이 없습니다.")

🔄 레시피 순서 통합 처리 중...
✅ recipe_steps 컬럼 생성 완료!

📋 첫 번째 레시피 예시 (새우 두부 계란찜):
1. 손질된 새우를 끓는 물에 데쳐 건진다.
2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.
3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분 정도 찐다.

📊 통계:
- 전체 레시피: 1,136개
- 조리순서가 있는 레시피: 1,135개
- 조리순서 비율: 99.9%


## 4. 💾 최종 데이터 저장

## 3. 🥕 재료 정보 구조화

In [117]:
# 기존 CSV 파일에서 재료 정보 구조화
import json

def parse_ingredient_text(ingredient_text):
    """
    재료 텍스트를 파싱하여 구조화된 정보 추출
    
    Args:
        ingredient_text: "연두부 75g(3/4모)" 형태의 텍스트
        
    Returns:
        dict: {'name': '연두부', 'amount': '75', 'unit': 'g', 'description': '3/4모'}
    """
    if not ingredient_text or not ingredient_text.strip():
        return None
    
    ingredient_text = ingredient_text.strip()
    
    # 기본 구조
    result = {
        'name': '',
        'amount': '',
        'unit': '',
        'description': ''
    }
    
    # 괄호 안의 설명 추출
    description_match = re.search(r'\(([^)]+)\)', ingredient_text)
    if description_match:
        result['description'] = description_match.group(1)
        ingredient_text = ingredient_text.replace(description_match.group(0), '').strip()
    
    # 숫자와 단위 추출 (예: "75g", "1큰술", "2/3작은술")
    amount_match = re.search(r'([0-9./]+)\s*([a-zA-Z가-힣]+)', ingredient_text)
    if amount_match:
        result['amount'] = amount_match.group(1)
        result['unit'] = amount_match.group(2)
        # 재료명은 숫자/단위 앞의 부분
        result['name'] = ingredient_text[:amount_match.start()].strip()
    else:
        # 숫자/단위가 없는 경우 전체를 재료명으로
        result['name'] = ingredient_text
    
    return result

def structure_ingredients(rcp_parts_dtls):
    """
    RCP_PARTS_DTLS 텍스트를 구조화된 재료 정보로 변환
    
    Args:
        rcp_parts_dtls: 원본 재료 정보 텍스트
        
    Returns:
        dict: 구조화된 재료 정보
    """
    if pd.isna(rcp_parts_dtls) or not str(rcp_parts_dtls).strip():
        return {'categories': []}
    
    lines = str(rcp_parts_dtls).strip().split('\n')
    categories = []
    current_category = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 콤마로 구분된 재료들이 있는지 확인
        if ',' in line:
            # 재료 리스트인 경우
            ingredients = []
            for ingredient in line.split(','):
                parsed = parse_ingredient_text(ingredient.strip())
                if parsed and parsed['name']:
                    ingredients.append(parsed)
            
            if current_category:
                current_category['ingredients'].extend(ingredients)
            else:
                # 카테고리 없이 재료만 있는 경우
                categories.append({
                    'category': '기본재료',
                    'ingredients': ingredients
                })
        else:
            # 카테고리명 또는 단일 재료인 경우
            parsed = parse_ingredient_text(line)
            if parsed and parsed['amount']:  # 수량이 있으면 재료
                if current_category:
                    current_category['ingredients'].append(parsed)
                else:
                    categories.append({
                        'category': '기본재료', 
                        'ingredients': [parsed]
                    })
            else:  # 수량이 없으면 카테고리명
                current_category = {
                    'category': line,
                    'ingredients': []
                }
                categories.append(current_category)
    
    return {'categories': categories}

# 현재 데이터에서 재료 정보 확인
if not df.empty and 'RCP_PARTS_DTLS' in df.columns:
    print("🔍 재료 정보 구조화 분석 중...")
    
    # 샘플 데이터 확인
    sample_ingredients = df['RCP_PARTS_DTLS'].iloc[0]
    print(f"\n📋 원본 재료 정보 (첫 번째 레시피):")
    print("=" * 60)
    print(sample_ingredients)
    
    # 구조화 테스트
    structured = structure_ingredients(sample_ingredients)
    print(f"\n🔧 구조화된 재료 정보:")
    print("=" * 60)
    print(json.dumps(structured, ensure_ascii=False, indent=2))
    
    # 몇 개 더 샘플 확인
    print(f"\n🔍 더 많은 샘플 확인:")
    print("=" * 60)
    for i in range(1, min(4, len(df))):
        print(f"\n레시피 {i+1}: {df.iloc[i]['RCP_NM']}")
        print(f"원본: {str(df.iloc[i]['RCP_PARTS_DTLS'])[:100]}...")
        structured_sample = structure_ingredients(df.iloc[i]['RCP_PARTS_DTLS'])
        print(f"카테고리 수: {len(structured_sample['categories'])}개")
        for cat in structured_sample['categories']:
            print(f"  - {cat['category']}: {len(cat['ingredients'])}개 재료")

else:
    print("❌ RCP_PARTS_DTLS 컬럼을 찾을 수 없습니다.")

🔍 재료 정보 구조화 분석 중...

📋 원본 재료 정보 (첫 번째 레시피):
새우두부계란찜
연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)
고명
시금치 10g(3줄기)

🔧 구조화된 재료 정보:
{
  "categories": [
    {
      "category": "새우두부계란찜",
      "ingredients": [
        {
          "name": "연두부",
          "amount": "75",
          "unit": "g",
          "description": "3/4모"
        },
        {
          "name": "칵테일새우",
          "amount": "20",
          "unit": "g",
          "description": "5마리"
        },
        {
          "name": "달걀",
          "amount": "30",
          "unit": "g",
          "description": "1/2개"
        },
        {
          "name": "생크림",
          "amount": "13",
          "unit": "g",
          "description": "1큰술"
        },
        {
          "name": "설탕",
          "amount": "5",
          "unit": "g",
          "description": "1작은술"
        },
        {
          "name": "무염버터",
          "amount": "5",
          "unit": "g",
          "description": "1작은술"
     

In [118]:
# 전체 데이터에 재료 구조화 적용 (개선된 버전)
def apply_ingredient_structuring_fixed(df):
    """
    DataFrame의 모든 레시피에 재료 구조화 적용 (JSON 포맷 개선)
    
    Args:
        df: 원본 DataFrame
        
    Returns:
        구조화된 재료 정보가 추가된 DataFrame
    """
    df_structured = df.copy()
    
    print("🔄 전체 레시피에 재료 구조화 적용 중 (개선된 버전)...")
    
    # 구조화된 재료 정보를 저장할 리스트
    structured_ingredients = []
    parsing_errors = []
    
    for idx, row in df.iterrows():
        try:
            structured = structure_ingredients(row['RCP_PARTS_DTLS'])
            # JSON으로 직렬화하되, ensure_ascii=False로 한글 보존
            json_str = json.dumps(structured, ensure_ascii=False, separators=(',', ':'))
            structured_ingredients.append(json_str)
            
            # 진행률 표시 (100개마다)
            if (idx + 1) % 100 == 0:
                print(f"✅ {idx + 1}/{len(df)} 처리 완료 ({((idx + 1)/len(df)*100):.1f}%)")
                
        except Exception as e:
            parsing_errors.append({'index': idx, 'recipe': row.get('RCP_NM', 'Unknown'), 'error': str(e)})
            structured_ingredients.append('{"categories":[]}')  # 빈 구조
    
    # 새 컬럼 추가
    df_structured['ingredients_structured'] = structured_ingredients
    
    print(f"\n✅ 재료 구조화 완료!")
    print(f"📊 총 처리된 레시피: {len(df)}개")
    print(f"❌ 파싱 오류: {len(parsing_errors)}개")
    
    if parsing_errors:
        print(f"\n⚠️ 파싱 오류 상위 5개:")
        for error in parsing_errors[:5]:
            print(f"  - 레시피 {error['index']}: {error['recipe']} - {error['error']}")
    
    return df_structured

# 기존 데이터에 재료 구조화 적용 (개선된 버전)
if not df.empty and 'RCP_PARTS_DTLS' in df.columns:
    df_with_fixed_ingredients = apply_ingredient_structuring_fixed(df)
    
    # 결과 확인
    print(f"\n🔍 개선된 구조화 결과 샘플:")
    print("=" * 80)
    
    # 첫 번째 레시피의 구조화된 재료 정보 확인
    sample_json = df_with_fixed_ingredients.iloc[0]['ingredients_structured']
    print(f"📋 레시피: {df_with_fixed_ingredients.iloc[0]['RCP_NM']}")
    print(f"🔧 JSON 길이: {len(sample_json)} 문자")
    print(f"📝 JSON 미리보기 (첫 200자):")
    print(sample_json[:200] + "...")
    
    # JSON 파싱 테스트
    try:
        parsed = json.loads(sample_json)
        print(f"\n✅ JSON 파싱 성공!")
        print(f"🥕 카테고리 수: {len(parsed['categories'])}개")
        
        for i, category in enumerate(parsed['categories']):
            print(f"  {i+1}. {category['category']}: {len(category['ingredients'])}개 재료")
    except Exception as e:
        print(f"\n❌ JSON 파싱 실패: {e}")
    
    # 데이터 업데이트
    df = df_with_fixed_ingredients
    
else:
    print("❌ 데이터가 없거나 RCP_PARTS_DTLS 컬럼을 찾을 수 없습니다.")

🔄 전체 레시피에 재료 구조화 적용 중 (개선된 버전)...
✅ 100/1136 처리 완료 (8.8%)
✅ 200/1136 처리 완료 (17.6%)
✅ 300/1136 처리 완료 (26.4%)
✅ 400/1136 처리 완료 (35.2%)
✅ 500/1136 처리 완료 (44.0%)
✅ 600/1136 처리 완료 (52.8%)
✅ 700/1136 처리 완료 (61.6%)
✅ 800/1136 처리 완료 (70.4%)
✅ 900/1136 처리 완료 (79.2%)
✅ 1000/1136 처리 완료 (88.0%)
✅ 1100/1136 처리 완료 (96.8%)

✅ 재료 구조화 완료!
📊 총 처리된 레시피: 1136개
❌ 파싱 오류: 0개

🔍 개선된 구조화 결과 샘플:
📋 레시피: 새우 두부 계란찜
🔧 JSON 길이: 512 문자
📝 JSON 미리보기 (첫 200자):
{"categories":[{"category":"새우두부계란찜","ingredients":[{"name":"연두부","amount":"75","unit":"g","description":"3/4모"},{"name":"칵테일새우","amount":"20","unit":"g","description":"5마리"},{"name":"달걀","amount":"30...

✅ JSON 파싱 성공!
🥕 카테고리 수: 2개
  1. 새우두부계란찜: 6개 재료
  2. 고명: 1개 재료


In [119]:
# 개선된 구조화 데이터 저장
def save_structured_data_fixed(df):
    """
    구조화된 재료 정보를 포함한 데이터를 CSV로 저장 (개선된 버전)
    
    Args:
        df: 구조화된 재료 정보가 포함된 DataFrame
    """
    if 'ingredients_structured' not in df.columns:
        print("❌ ingredients_structured 컬럼이 없습니다.")
        return
    
    # 파일 저장 경로 설정
    data_dir = r'C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data'
    os.makedirs(data_dir, exist_ok=True)
    
    # 구조화된 데이터 저장 (개선된 버전)
    structured_file = os.path.join(data_dir, '레시피_구조화_완료_개선.csv')
    
    # UTF-8 인코딩으로 저장하여 한글 문제 해결
    df.to_csv(structured_file, index=False, encoding='utf-8-sig', quoting=1)
    
    print(f"💾 개선된 구조화 데이터 저장 완료!")
    print(f"📁 파일 경로: {structured_file}")
    print(f"📊 저장된 데이터: {len(df):,}개 레시피, {len(df.columns)}개 컬럼")
    
    # 저장된 파일의 JSON 형식 검증
    print(f"\n🔍 저장된 파일 검증 중...")
    try:
        # 파일을 다시 읽어서 JSON 파싱 테스트
        df_test = pd.read_csv(structured_file, encoding='utf-8-sig')
        sample_json = df_test['ingredients_structured'].iloc[0]
        parsed = json.loads(sample_json)
        print(f"✅ 저장된 파일의 JSON 형식 정상!")
        print(f"📋 테스트 파싱 결과: {len(parsed['categories'])}개 카테고리")
        
        # 몇 개 샘플 확인
        for i in range(min(3, len(df_test))):
            test_json = df_test['ingredients_structured'].iloc[i]
            try:
                test_parsed = json.loads(test_json)
                print(f"  레시피 {i+1}: ✅ JSON 파싱 성공 ({len(test_parsed['categories'])}개 카테고리)")
            except Exception as e:
                print(f"  레시피 {i+1}: ❌ JSON 파싱 실패 - {e}")
        
    except Exception as e:
        print(f"❌ 파일 검증 실패: {e}")
    
    # 추가 분석 결과 저장
    analysis_results = analyze_structured_ingredients(df)
    
    # 분석 결과를 JSON으로 저장
    analysis_file = os.path.join(data_dir, '재료_분석_결과_개선.json')
    with open(analysis_file, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)
    
    print(f"📈 분석 결과 저장: {analysis_file}")
    
    return structured_file, analysis_file

# 개선된 구조화 저장 실행
if not df.empty and 'ingredients_structured' in df.columns:
    structured_file, analysis_file = save_structured_data_fixed(df)
    
    # 분석 결과 출력
    print(f"\n📊 재료 분석 결과 (개선된 버전):")
    print("=" * 80)
    
    with open(analysis_file, 'r', encoding='utf-8') as f:
        analysis_results = json.load(f)
    
    print(f"📈 평균 재료 수: {analysis_results['평균_재료_수']}개")
    
    print(f"\n🏷️ 상위 카테고리 (상위 5개):")
    for category, count in list(analysis_results['카테고리_통계'].items())[:5]:
        print(f"  - {category}: {count}회")
    
    print(f"\n🥕 상위 재료 (상위 10개):")
    for ingredient, count in list(analysis_results['재료_통계'].items())[:10]:
        print(f"  - {ingredient}: {count}회")
    
    print(f"\n📏 상위 단위 (상위 5개):")
    for unit, count in list(analysis_results['단위_통계'].items())[:5]:
        print(f"  - {unit}: {count}회")

else:
    print("❌ 구조화된 재료 정보가 없습니다. 먼저 구조화를 실행해주세요.")

💾 개선된 구조화 데이터 저장 완료!
📁 파일 경로: C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data\레시피_구조화_완료_개선.csv
📊 저장된 데이터: 1,136개 레시피, 57개 컬럼

🔍 저장된 파일 검증 중...
✅ 저장된 파일의 JSON 형식 정상!
📋 테스트 파싱 결과: 2개 카테고리
  레시피 1: ✅ JSON 파싱 성공 (2개 카테고리)
  레시피 2: ✅ JSON 파싱 성공 (2개 카테고리)
  레시피 3: ✅ JSON 파싱 성공 (2개 카테고리)
📈 분석 결과 저장: C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data\재료_분석_결과_개선.json

📊 재료 분석 결과 (개선된 버전):
📈 평균 재료 수: 11.12개

🏷️ 상위 카테고리 (상위 5개):
  - 기본재료: 2424회
  - 소금(0.2g): 4회
  - 참기름(5g): 3회
  - 소금(0.3g): 3회
  - 후춧가루(0.01g): 3회

🥕 상위 재료 (상위 10개):
  - 양파: 433회
  - 소금: 308회
  - 설탕: 269회
  - 마늘: 240회
  - 물: 227회
  - 당근: 227회
  - 후춧가루: 183회
  - 식초: 176회
  - 참기름: 175회
  - 달걀: 158회

📏 상위 단위 (상위 5개):
  - g: 5546회
  - ml: 62회
  - 인분: 61회
  - 개: 19회
  - cm: 8회


In [120]:
# 기존 저장된 CSV 파일 로드 (재료 구조화를 위해)
import os

# CSV 파일 경로
csv_file_path = r'C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data\레시피_통합본.csv'

if os.path.exists(csv_file_path):
    print(f"📂 기존 CSV 파일 로드 중...")
    df_from_csv = pd.read_csv(csv_file_path)
    print(f"✅ CSV 파일 로드 완료! 크기: {df_from_csv.shape}")
    
    # 현재 노트북의 df와 비교
    if 'df' in locals() and not df.empty:
        print(f"📊 현재 노트북 df: {df.shape}")
        print(f"📊 CSV 파일 df: {df_from_csv.shape}")
        
        # CSV 파일을 우선 사용 (recipe_steps 컬럼이 있는지 확인)
        if 'recipe_steps' in df_from_csv.columns:
            print("🔄 CSV 파일을 주 데이터로 사용합니다.")
            df = df_from_csv
        else:
            print("⚠️ CSV 파일에 recipe_steps 컬럼이 없습니다. 노트북 df를 사용합니다.")
    else:
        print("🔄 CSV 파일을 주 데이터로 사용합니다.")
        df = df_from_csv
        
    print(f"\\n📋 현재 사용 중인 데이터: {df.shape}")
    print(f"💡 컬럼 목록: {list(df.columns)}")
    
else:
    print(f"❌ CSV 파일을 찾을 수 없습니다: {csv_file_path}")
    print("💡 먼저 앞의 셀들을 실행하여 데이터를 생성해주세요.")

📂 기존 CSV 파일 로드 중...
✅ CSV 파일 로드 완료! 크기: (1136, 13)
📊 현재 노트북 df: (1136, 57)
📊 CSV 파일 df: (1136, 13)
🔄 CSV 파일을 주 데이터로 사용합니다.
\n📋 현재 사용 중인 데이터: (1136, 13)
💡 컬럼 목록: ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'RCP_PARTS_DTLS', 'recipe_steps', 'RCP_NA_TIP', 'HASH_TAG']


In [121]:
# 통합된 데이터를 CSV로 저장
if not df.empty and 'recipe_steps' in df.columns:
    # 주요 컬럼들만 선택해서 저장
    selected_columns = [
        'RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 
        'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA',
        'RCP_PARTS_DTLS', 'recipe_steps', 'RCP_NA_TIP', 'HASH_TAG'
    ]
    
    # 존재하는 컬럼만 필터링
    available_columns = [col for col in selected_columns if col in df.columns]
    df_final = df[available_columns]
    
    # 파일 저장 경로 설정
    import os
    data_dir = r'C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data'
    os.makedirs(data_dir, exist_ok=True)
    
    # CSV 파일로 저장
    output_file = os.path.join(data_dir, '레시피_통합본.csv')
    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"💾 {output_file} 파일로 저장 완료!")
    print(f"📊 저장된 데이터: {len(df_final):,}개 레시피, {len(df_final.columns)}개 컬럼")
    print(f"📝 포함된 컬럼: {list(df_final.columns)}")
    
    # 샘플 데이터 미리보기
    print(f"\n🔍 데이터 미리보기:")
    print("=" * 80)
    sample_recipe = df_final.iloc[0]
    print(f"📋 레시피명: {sample_recipe['RCP_NM']}")
    print(f"🍳 조리방법: {sample_recipe['RCP_WAY2']}")
    print(f"🥘 요리종류: {sample_recipe['RCP_PAT2']}")
    print(f"⚡ 칼로리: {sample_recipe['INFO_ENG']}kcal")
    
    print(f"\n🥕 재료:")
    ingredients_text = str(sample_recipe['RCP_PARTS_DTLS'])
    if len(ingredients_text) > 200:
        print(ingredients_text[:200] + "...")
    else:
        print(ingredients_text)
        
    print(f"\n👩‍🍳 조리순서:")
    print(sample_recipe['recipe_steps'])
    
    # 통계 정보
    print(f"\n📊 최종 통계:")
    print("=" * 50)
    print(f"📈 총 레시피 수: {len(df_final):,}개")
    print(f"✅ recipe_steps가 있는 레시피: {df_final['recipe_steps'].notna().sum():,}개")
    print(f"❌ recipe_steps가 비어있는 레시피: {(df_final['recipe_steps'] == '').sum():,}개")
    
    # 요리 방법별 분포
    print(f"\n🍳 요리 방법별 분포 (상위 5개):")
    cooking_methods = df_final['RCP_WAY2'].value_counts()
    for method, count in cooking_methods.head(5).items():
        print(f"  {method}: {count:,}개")
    
    # 요리 종류별 분포  
    print(f"\n🥘 요리 종류별 분포 (상위 5개):")
    cooking_types = df_final['RCP_PAT2'].value_counts()
    for cook_type, count in cooking_types.head(5).items():
        print(f"  {cook_type}: {count:,}개")
    
else:
    print("❌ recipe_steps 컬럼이 없습니다. 먼저 레시피 순서를 통합해주세요.")

💾 C:\Users\user\Desktop\SKN_AFTER_STUDY\project1\data\레시피_통합본.csv 파일로 저장 완료!
📊 저장된 데이터: 1,136개 레시피, 13개 컬럼
📝 포함된 컬럼: ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'RCP_PARTS_DTLS', 'recipe_steps', 'RCP_NA_TIP', 'HASH_TAG']

🔍 데이터 미리보기:
📋 레시피명: 새우 두부 계란찜
🍳 조리방법: 찌기
🥘 요리종류: 반찬
⚡ 칼로리: 220.0kcal

🥕 재료:
새우두부계란찜
연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생크림 13g(1큰술), 설탕 5g(1작은술), 무염버터 5g(1작은술)
고명
시금치 10g(3줄기)

👩‍🍳 조리순서:
1. 손질된 새우를 끓는 물에 데쳐 건진다.
2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.
3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분 정도 찐다.

📊 최종 통계:
📈 총 레시피 수: 1,136개
✅ recipe_steps가 있는 레시피: 1,135개
❌ recipe_steps가 비어있는 레시피: 0개

🍳 요리 방법별 분포 (상위 5개):
  끓이기: 325개
  기타: 292개
  굽기: 259개
  볶기: 108개
  찌기: 82개

🥘 요리 종류별 분포 (상위 5개):
  반찬: 574개
  일품: 171개
  후식: 132개
  밥: 119개
  국&찌개: 103개


## 5. 🎯 작업 완료 요약

In [122]:
# 🎯 작업 완료 요약
print("="*80)
print("🎉 레시피 데이터 처리 및 구조화 완료!")
print("="*80)
print("""
✅ 완료된 작업:
1. 🌐 식품안전나라 API에서 1,136개 레시피 데이터 수집
2. 🗂️ DataFrame으로 변환 및 컬럼 논리적 순서 정리
3. 🔧 MANUAL01~MANUAL20 컬럼을 하나의 'recipe_steps' 컬럼으로 통합
4. ✏️ 레시피 순서를 '1. 씻는다, 2. 손질한다, 3. 썬다' 형태로 정리
5. 🥕 재료 정보(RCP_PARTS_DTLS)를 딕셔너리+리스트로 구조화
6. 📊 재료 통계 분석 및 인사이트 도출
7. 💾 구조화된 데이터를 CSV와 JSON으로 저장

📁 생성된 파일:
- 📋 레시피_통합본.csv: 기본 통합 데이터 (13개 컬럼)
- 🗂️ 레시피_구조화_완료.csv: 재료 구조화 포함 (14개 컬럼)
- 📈 재료_분석_결과.json: 재료 통계 분석 결과

📊 데이터 구조:
- 🍽️ 레시피명, 조리방법, 요리종류, 영양정보
- 🥕 원본 재료 정보 (RCP_PARTS_DTLS)
- 🗂️ 구조화된 재료 정보 (ingredients_structured)
  └── 카테고리별 재료 리스트 (이름, 수량, 단위, 설명)
- 👩‍🍳 통합된 조리순서 (recipe_steps)
- 💡 팁 및 해시태그

🎯 구조화된 재료 정보 예시:
{
  "categories": [
    {
      "category": "주재료",
      "ingredients": [
        {"name": "연두부", "amount": "75", "unit": "g", "description": "3/4모"}
      ]
    }
  ]
}

🚀 이제 레시피 데이터가 완전히 구조화되어 다음 작업에 활용할 수 있습니다:
   - 재료별 레시피 검색 및 추천
   - 영양성분 분석 및 계산
   - 레시피 유사도 분석
   - 머신러닝 모델 학습용 데이터
""")
print("="*80)

🎉 레시피 데이터 처리 및 구조화 완료!

✅ 완료된 작업:
1. 🌐 식품안전나라 API에서 1,136개 레시피 데이터 수집
2. 🗂️ DataFrame으로 변환 및 컬럼 논리적 순서 정리
3. 🔧 MANUAL01~MANUAL20 컬럼을 하나의 'recipe_steps' 컬럼으로 통합
4. ✏️ 레시피 순서를 '1. 씻는다, 2. 손질한다, 3. 썬다' 형태로 정리
5. 🥕 재료 정보(RCP_PARTS_DTLS)를 딕셔너리+리스트로 구조화
6. 📊 재료 통계 분석 및 인사이트 도출
7. 💾 구조화된 데이터를 CSV와 JSON으로 저장

📁 생성된 파일:
- 📋 레시피_통합본.csv: 기본 통합 데이터 (13개 컬럼)
- 🗂️ 레시피_구조화_완료.csv: 재료 구조화 포함 (14개 컬럼)
- 📈 재료_분석_결과.json: 재료 통계 분석 결과

📊 데이터 구조:
- 🍽️ 레시피명, 조리방법, 요리종류, 영양정보
- 🥕 원본 재료 정보 (RCP_PARTS_DTLS)
- 🗂️ 구조화된 재료 정보 (ingredients_structured)
  └── 카테고리별 재료 리스트 (이름, 수량, 단위, 설명)
- 👩‍🍳 통합된 조리순서 (recipe_steps)
- 💡 팁 및 해시태그

🎯 구조화된 재료 정보 예시:
{
  "categories": [
    {
      "category": "주재료",
      "ingredients": [
        {"name": "연두부", "amount": "75", "unit": "g", "description": "3/4모"}
      ]
    }
  ]
}

🚀 이제 레시피 데이터가 완전히 구조화되어 다음 작업에 활용할 수 있습니다:
   - 재료별 레시피 검색 및 추천
   - 영양성분 분석 및 계산
   - 레시피 유사도 분석
   - 머신러닝 모델 학습용 데이터



In [123]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RCP_SEQ         1136 non-null   int64  
 1   RCP_NM          1136 non-null   object 
 2   RCP_WAY2        1136 non-null   object 
 3   RCP_PAT2        1136 non-null   object 
 4   INFO_ENG        1136 non-null   float64
 5   INFO_CAR        1136 non-null   float64
 6   INFO_PRO        1136 non-null   float64
 7   INFO_FAT        1136 non-null   float64
 8   INFO_NA         1136 non-null   float64
 9   RCP_PARTS_DTLS  1133 non-null   object 
 10  recipe_steps    1135 non-null   object 
 11  RCP_NA_TIP      1082 non-null   object 
 12  HASH_TAG        357 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 115.5+ KB
